In [90]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import random_graph
import networkx as nx
import matplotlib.pyplot as plt
import uuid
import pickle
from scipy.signal import convolve
import os
from tqdm import tqdm

In [2]:
rng = np.random.default_rng()

In [3]:
# Note -- place your own MNIST files in the appropriate directory
train_data = np.loadtxt("./data/mnist/mnist_train.csv", delimiter=',')
test_data = np.loadtxt("./data/mnist/mnist_test.csv", delimiter=',')

In [4]:
train_imgs = train_data[:, 1:]  # (60000, 784)
test_imgs = test_data[:, 1:]  # (10000, 784)
train_labels = train_data[:, 0]  # (60000, )
test_labels = test_data[:, 0]  # (10000, )

In [5]:
# Change the top k input values to 1, rest of the values to 0
def k_cap(input, cap_size):
    output = np.zeros_like(input)
    if len(input.shape) == 1:
        idx = np.argsort(input)[-cap_size:]
        output[idx] = 1
    else:
        idx = np.argsort(input, axis=-1)[:, -cap_size:]
        np.put_along_axis(output, idx, 1, axis=-1)
    return output

In [6]:
# EXPERIMENT_STORE = []
# ID_SET = set()

# with open('experiment_store.pickle', 'wb') as f:
#     pickle.dump(EXPERIMENT_STORE, f)

# with open('id_set.pickle', 'wb') as f:
#     pickle.dump(ID_SET, f)

In [7]:
with open('experiment_store.pickle', 'rb') as f:
    EXPERIMENT_STORE = pickle.load(f)

with open('id_set.pickle', 'rb') as f:
    ID_SET = pickle.load(f)

In [86]:
# sample a simple graph, approximately uniformly at random, from all graphs with given degree sequence
# MCMC occurs under the hood

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1, keepdims=True)

def run_experiment(train_imgs, test_imgs, train_labels, test_labels, verbose=True, **kwargs):
    
    # Sensory area graph
    # Runnable Object for each model

    n_in = kwargs.get('n_in', 784)
    n_neurons = kwargs.get('n_neurons', 2000)
    n_out = n_neurons
    cap_size = kwargs.get('cap_size', 200)
    sparsity = kwargs.get('sparsity', 0.1) # Not Used
    n_rounds = kwargs.get('n_rounds', 5)
    beta = kwargs.get('beta', 1e0)
    degree_sequence_A = kwargs.get('degree_sequence_A', (200,) * 10 + (100,) * 25 + (50,) * 50 + (25,) * 100 + (20,) * 250 + (10,) * 400 + (5,) * 500 + (1,) * 664)
    degree_sequence_W = kwargs.get('degree_sequence_W', (200,) * 10 + (100,) * 25 + (50,) * 50 + (25,) * 100 + (20,) * 250 + (10,) * 400 + (5,) * 500 + (1,) * 664)

    experiment_id = uuid.uuid1()
    while experiment_id in ID_SET:
        experiment_id = uuid.uuid1()
    
    if not os.path.exists('./results'):
        os.mkdir('./results')

    if not os.path.exists(f'./results/{experiment_id}'):
        os.mkdir(f'./results/{experiment_id}')

    A_edges = random_graph.sample_simple_graph(degree_sequence_A)
    W_edges = random_graph.sample_simple_graph(degree_sequence_W)

    A_graph = nx.DiGraph()
    A_graph.add_nodes_from(range(n_neurons))
    A_graph.add_edges_from(A_edges)

    W_graph = nx.DiGraph()
    W_graph.add_nodes_from(range(n_neurons))
    W_graph.add_edges_from(W_edges)

    if verbose:
        nx.draw(W_graph)
        plt.savefig(f'./results/{experiment_id}/graph_visualization.png')
    
    W = np.squeeze(np.asarray((nx.linalg.graphmatrix.adjacency_matrix(W_graph).todense() & np.logical_not(np.eye(n_neurons, dtype=bool)))))
    A = np.squeeze(np.asarray((nx.linalg.graphmatrix.adjacency_matrix(A_graph).todense() & np.logical_not(np.eye(n_neurons, dtype=bool)))))

    W = W.astype(np.float64)
    A = A.astype(np.float64)[:n_in, :n_neurons]

    # k-cap on convolved input pixels
    n_examples = 5000
    examples = np.zeros((10, n_examples, 784))
    for i in range(10):
        examples[i] = k_cap(convolve(train_imgs[train_labels == i][:n_examples].reshape(-1, 28, 28), np.ones((1, 3, 3)), mode='same').reshape(-1, 28 * 28), cap_size)

    # Init connections from each neuron to sum up to 1
    W = np.ones_like(W)
    A = np.ones_like(A)
    W /= W.sum(axis=0, keepdims=True)
    A /= A.sum(axis=0, keepdims=True)
    bias = np.zeros(n_neurons)
    b = -1
    activations = np.zeros((10, n_rounds, n_neurons))

    # Loop over each class
    for i in range(10):
        act_h = np.zeros(n_neurons)
        
        # Loop over several examples
        for j in range(n_rounds):
            input = examples[i, j]
            
            # calculate activation
            # print(W.shape)
            # print(A.shape)
            act_h_new = k_cap(act_h @ W + input @ A + bias, cap_size)
            activations[i, j] = act_h_new.copy()
            
            # update weights
            A[(input > 0)[:, np.newaxis] & (act_h_new > 0)[np.newaxis, :]] *= 1 + beta
            W[(act_h > 0)[:, np.newaxis] & (act_h_new > 0)[np.newaxis, :]] *= 1 + beta
            
            act_h = act_h_new
            
        bias[act_h > 0] += b
        A /= A.sum(axis=0, keepdims=True)
        W /= W.sum(axis=0, keepdims=True)


    outputs = np.zeros((10, n_rounds+1, n_examples, n_neurons))
    for i in np.arange(10):
        # Run each example through the model n_round times
        for j in range(n_rounds):
            outputs[i, j+1] = k_cap(outputs[i, j] @ W + examples[i] @ A, cap_size)

    idx = np.full(n_neurons, -1, dtype=int)
    act = activations[:, -1].copy()       # final state activation after training each class
    act.shape

    for i, j in enumerate(range(10)):
        idx[i*cap_size:(i+1)*cap_size] = act[j].argsort()[-cap_size:][::-1]
        act[:, idx[i*cap_size:(i+1)*cap_size]] = -1
    
    r = np.arange(n_neurons)
    r[idx[idx > -1]] = -1
    idx[(i+1)*cap_size:] = np.unique(r)[1:]

    if verbose:
        get_ipython().run_line_magic('matplotlib', 'inline')

        fig, axes = plt.subplots(10, n_rounds, figsize=(10, 2 * 10), sharex=True, sharey=True)
        for ax, output in zip(axes, outputs):
            for i in range(n_rounds):
                ax[i].imshow((output[i+1] > 0)[:n_neurons, idx])
                ax[i].set_axis_off()
        fig.text(0.5, 0.04, 'Neurons', ha='center', va='center')
        fig.text(0.04, 0.5, 'Samples', ha='center', va='center', rotation='vertical')
        plt.savefig(f'./results/{experiment_id}/neuron_activation_grid.png')
    
    v = 0.1 * rng.standard_normal((10, n_neurons))
    targets = np.zeros((100, 10))

    for i in range(10):
        targets[i*10:(i+1)*10, i] = 1
    update = np.zeros_like(v)

    for z in range(10):
        permutation = rng.permutation(n_examples - 1000)
        # print(f'{z}th iteration')

    for j in range((n_examples - 1000) // 10):
        batch = outputs[:, 1, permutation[j*10:(j+1)*10]].reshape(10 * 10, n_neurons)
        scores = softmax((batch[:, :, np.newaxis] * v.T[np.newaxis, :, :]).sum(axis=1))
        update = 0.5 * update + 1e-3 * (batch[:, np.newaxis, :] * (scores - targets)[:, :, np.newaxis]).sum(axis=0)
        v -= update

    if verbose:
        fig, ax = plt.subplots(figsize=(10, 4))
        for i in range(10):
            # Pass each sample to the model and get its result 
            ax.bar(np.arange(n_neurons), outputs[i, -1].mean(axis=0)[idx], label=i)
        ax.legend(loc='upper right', ncol=2)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_ylim([0, 1.1])
        ax.set_xticklabels([])
        ax.set_xlabel('Neurons')
        ax.set_ylabel('Firing Probability')
    
    plt.savefig(f'./results/{experiment_id}/firing_probability.png')

    # c is a mask for identifying each assembly
    # set top k neurons to value 1 and 0 otherwise 
    c = np.zeros((10, n_neurons))
    use_train_act = True

    for i in range(10):
        
        if use_train_act:
            # create mask based on the last activation of each class during training
            c[i, activations[i, -1].argsort()[-cap_size:]] = 1
        else:
            # create mask based on the activation after 1 round of ALL the samples for each class
            c[i, outputs[i, 1].sum(axis=0).argsort()[-cap_size:]] = 1
            

    if verbose:
        fig, axes = plt.subplots(1, 10, figsize=(10, 2))
        for i in range(10):
            axes[i].imshow((A * c[i][np.newaxis, :]).sum(axis=1).reshape(28, 28))
            axes[i].set_axis_off()
        fig.tight_layout()
    
    plt.savefig(f'./results/{experiment_id}/digit_representation.png')

 
    predictions = (outputs[:, 1] @ c.T).argmax(axis=-1)
    acc = (predictions == np.arange(10)[:, np.newaxis]).sum(axis=-1) / n_examples
    
    results = {
        'experiment_id': experiment_id,
        'parameters': {
            'degree_distribution': degree_sequence
        },
        'training_first_n': ((outputs[:, 1, :-1000] @ v.T).argmax(axis=-1) == np.arange(10)[:, np.newaxis]).sum() / 40000,
        'training_last_n': ((outputs[:, 1, -1000:] @ v.T).argmax(axis=-1) == np.arange(10)[:, np.newaxis]).sum() / 10000,
        'acc': acc, 
        'acc_mean': acc.mean(),
    }

    EXPERIMENT_STORE.append(results)

    with open('experiment_store.pickle', 'wb') as f:
        pickle.dump(EXPERIMENT_STORE, f)

    with open('id_set.pickle', 'wb') as f:
        pickle.dump(ID_SET, f)

    return results


## Manually defined degree sequences experiment

In [87]:
# # degree sequence determines family we will sample from
# degree_sequence = (200,) * 10 + (100,) * 25 + (50,) * 50 + (25,) * 100 + (20,) * 250 + (10,) * 400 + (5,) * 500 + (1,) * 664  # 2550 edges in total
# results = run_experiment(train_imgs, test_imgs, train_labels, test_labels, verbose=True)

In [106]:
degree_sequences = [
    list((np.array((200,) * 10 + (100,) * 25 + (50,) * 50 + (25,) * 100 + (20,) * 250 + (10,) * 400 + (5,) * 500 + (1,) * 665)/2).round().astype(int)),
    list(np.array((100,) * 8 + (80,) * 12 + (50,) * 20 + (30,) * 30 + (20,) * 50 + (15,) * 75 + (10,) * 100 + (5,) * 200 + (2,)*600 + (1,) * 905))
]

In [ ]:
# from scipy.stats import beta
# for n_neurons in range(500, 4000, 500):
#     for a in [0.5, 1, 2]:
#         for b in [0.5, 1, 2]:
#             degree_distribution_samples = np.array(beta.rvs(a, b, size=n_neurons))
#             original_cardinality = sum(degree_distribution_samples)
#             degree_distribution_samples_transformed = (degree_distribution_samples*n_neurons/original_cardinality*50).round()
#             print(len(degree_distribution_samples_transformed))
#             print(sum(degree_distribution_samples_transformed))
#             print(degree_distribution_samples_transformed)
#             break
#         break 
#     break

500
25014.0
[ 99. 101.   2.   2. 103.   2.  44.  24.  57.  32.  81.  99.  91.  12.
   7.  90.   1.  80.  75.  40.  87.  27.  77.  10. 101.  17.  86.   1.
   2.  83.  92.  48.  47.  70.   0.  82.  64.  87.   1.  15. 101.  95.
  41.  52.  97.  62.   7.  86.  13.  51.  21.  12.  18.  74.   7.   1.
  64.  80.  15.  26.   8.  84.  89.  89.  20.  80.  54.   1.  65.  99.
  76.   5.  11.  88.  68.  96.  13.  60.  62.  62.  69.  93.   7.  51.
   3.  87.  11.  51.  60. 103.  71.   2. 102.  79.  87.  45.  28.  20.
   4.   1.  68.  82.  91.  68.   7.  28.   7.   9.  45.  67.  80.  20.
  13.  20.   0.  10.  59. 102. 101.  17.   3.   0.  94.   1. 101.  23.
  40.  71.  80.  82.  82.  19.  32.   2. 101.  61.  97.   4.   2.  23.
  99.  32.  93.  74.  10.  90. 102.   4.  65. 102.  33.  17.   1. 101.
  77.  15.  22. 103.  87.  23.  13.  21.  30.  33.   1.   2.  12.  92.
 102.   0. 102.  99.  29.  10.  20.   1.  55.  88.  93.  93. 100.  17.
  51.  17.  53.  45.   9.  27.   6.  52.  83.  25.  58.  28.  69.

In [108]:
iteration_list = []
for degree_sequence_W in degree_sequences:
    for scaling_factor_W in range(1, 3):
        iteration_list.append({
            'degree_sequence': degree_sequence_W,
            'scaling_factor_W': scaling_factor_W,
        })
for params in tqdm(iteration_list):
    scaling_factor_W = params['scaling_factor_W']
    degree_sequence_W = params['degree_sequence']
    final_degree_sequence_W = list(np.array(degree_sequence_W) * scaling_factor_W)
    run_experiment(train_imgs, test_imgs, train_labels, test_labels, verbose=True, degree_sequence_W=final_degree_sequence_W, degree_sequence_A=final_degree_sequence_W, n_neurons=len(final_degree_sequence_W))

  0%|          | 0/4 [00:09<?, ?it/s]


KeyboardInterrupt: 

## Distribution generated degree sequences experiment

In [ ]:
from scipy.stats import beta
from scipy.stats import uniform
from scipy.stats import binom

In [110]:
def generate_degree_sequences(distribution, n_neurons, desired_connections, verbose=False, **kwargs):
    samples_transformed = None
    if distribution == 'beta':
        if 'a' not in kwargs.keys() or 'b' not in kwargs.keys():
            raise ValueError('Need alpha or beta args')
        
        samples = beta.rvs(kwargs['a'], kwargs['b'], size=n_neurons)
        normalized_samples = samples/sum(samples)
        samples_transformed = (normalized_samples*desired_connections).round()
    elif distribution == 'uniform':
        samples = uniform.rvs(size=n_neurons)
        normalized_samples = samples/sum(samples)
        samples_transformed = (normalized_samples*desired_connections).round()
    elif distribution == 'binomial':
        if 'n' not in kwargs.keys() or 'p' not in kwargs.keys():
            raise ValueError('Need n or p args')

        samples = binom.rvs(n=kwargs['n'], p=kwargs['p'], size=n_neurons)
        normalized_samples = samples/sum(samples)
        samples_transformed = (normalized_samples*desired_connections).round()
    else:
        raise NotImplementedError()

    if samples_transformed is not None:
        if verbose:
            plt.hist(samples_transformed, density=True, histtype='stepfilled', alpha=0.2)
            plt.show()
        return list(samples_transformed.astype(np.int32))
    else:
        raise ValueError('Samples transformed is empty')


In [111]:
example_beta_degree_sequences = generate_degree_sequences('beta', 1999, 399800, a=0.4, b=0.4, verbose=True)
print(sum(example_beta_degree_sequences))
print(max(example_beta_degree_sequences))
print(sum(example_beta_degree_sequences)/len(example_beta_degree_sequences))
print() 


example_uniform_degree_sequences = generate_degree_sequences('uniform', 1999, 399800, verbose=True)
print(sum(example_uniform_degree_sequences))
print(max(example_uniform_degree_sequences))
print(sum(example_uniform_degree_sequences)/len(example_uniform_degree_sequences))
print() 


example_binomial_degree_sequences = generate_degree_sequences('binomial', 1999, 399800, n=5000, p=0.1, verbose=True)
print(sum(example_binomial_degree_sequences))
print(max(example_binomial_degree_sequences))
print(sum(example_binomial_degree_sequences)/len(example_binomial_degree_sequences))
print() 

example_degree_sequences = [example_uniform_degree_sequences, example_binomial_degree_sequences, example_beta_degree_sequences]

<IPython.core.display.Javascript object>

399802
405
200.00100050025011

399802
393
200.00100050025011

399818
227
200.00900450225112



In [113]:
iteration_list = []
for degree_sequence_W in example_degree_sequences:
    for scaling_factor_W in [0.5, 1, 1.5]:
        iteration_list.append({
            'degree_sequence': degree_sequence_W,
            'scaling_factor_W': scaling_factor_W,
        })

for params in tqdm(iteration_list):
    scaling_factor_W = params['scaling_factor_W']
    degree_sequence_W = params['degree_sequence']
    final_degree_sequence_W = list((np.array(degree_sequence_W) * scaling_factor_W).astype(np.int32))
    run_experiment(train_imgs, test_imgs, train_labels, test_labels, verbose=True, degree_sequence_W=final_degree_sequence_W, degree_sequence_A=final_degree_sequence_W, n_neurons=len(final_degree_sequence_W))

  0%|          | 0/9 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [ ]:
EXPERIMENT_STORE

[{'experiment_id': UUID('030704ac-583b-11ec-9c8f-a683e7c9b40b'),
  'parameters': {'degree_distribution': [100,
    100,
    100,
    100,
    100,
    100,
    100,
    100,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
  

In [ ]:
for experiment in EXPERIMENT_STORE:
    print(np.array(experiment['parameters']['degree_distribution']))
    print(experiment['acc'])
    print(experiment['acc_mean'])

[100 100 100 ...   1   1   1]
[0.8628 0.4354 0.466  0.7588 0.5334 0.257  0.8898 0.5568 0.5216 0.5452]
0.5826800000000001
[100 100 100 ...   1   1   1]
[0.8628 0.4354 0.466  0.7588 0.5334 0.257  0.8898 0.5568 0.5216 0.5452]
0.5826800000000001
[100 100 100 ...   1   1   1]
[0.8628 0.4354 0.466  0.7588 0.5334 0.257  0.8898 0.5568 0.5216 0.5452]
0.5826800000000001
[100 100 100 ...   1   1   1]
[0.8628 0.4354 0.466  0.7588 0.5334 0.257  0.8898 0.5568 0.5216 0.5452]
0.5826800000000001


In [ ]:
EXPERIMENT_STORE

[{'experiment_id': UUID('030704ac-583b-11ec-9c8f-a683e7c9b40b'),
  'parameters': {'degree_distribution': [100,
    100,
    100,
    100,
    100,
    100,
    100,
    100,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    80,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    50,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    30,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
  